In [ ]:
import plotly.io as pio

pio.renderers.default = "vscode+jupyterlab+notebook_connected"

: 

# Project 1 
### Analyzing NYC municipal employee payroll data

## Part 0: Read the data

Each record represents the following statistics for **every city employee**: Agency, Last Name, First Name, Middle Initial, Agency Start Date, Work Location Borough, Job Title Description, Leave Status as of the close of the FY (June 30th), Base Salary, Pay Basis, Regular Hours Paid, Regular Gross Paid, Overtime Hours worked, Total Overtime Paid, and Total Other Compensation (i.e. lump sum and/or retro payments). 

For this project, I will investigate the actual **base salary** paid to the municipal employees at **per annum basis** in fiscal year 2024, stored as `Regular Gross Paid`.

In [77]:
import pandas as pd

payroll = pd.read_csv('Citywide_Payroll_Data_2024.csv')
payroll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562898 entries, 0 to 562897
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Fiscal Year                 562898 non-null  int64  
 1   Payroll Number              562898 non-null  int64  
 2   Agency Name                 562898 non-null  object 
 3   Last Name                   562482 non-null  object 
 4   First Name                  562470 non-null  object 
 5   Mid Init                    327420 non-null  object 
 6   Agency Start Date           562821 non-null  object 
 7   Work Location Borough       562897 non-null  object 
 8   Title Description           562830 non-null  object 
 9   Leave Status as of June 30  562898 non-null  object 
 10  Base Salary                 562898 non-null  float64
 11  Pay Basis                   562898 non-null  object 
 12  Regular Hours               562898 non-null  float64
 13  Regular Gross 

In [78]:
# For convenience of the project, I will construct a new dataframe with colukn of interest.
# Clean the columns of interest. Make sure regular hours and regular gross paid are positive numbers.
df = payroll[payroll['Pay Basis']=='per Annum']
df = df[['Last Name', 'First Name', 'Work Location Borough', 'Title Description', 'Regular Hours', 'Regular Gross Paid']]
df = df[(df['Regular Hours'] > 0) & (df['Regular Gross Paid'] > 0)]
df.head()

,Last Name,First Name,Work Location Borough,Title Description,Regular Hours,Regular Gross Paid
1,OGUNLEYE,OLUKAYODE,BROOKLYN,CHILD PROTECTIVE SPECIALIST,1890.0,68022.06
2,LANE,SHIREE,QUEENS,CHILD PROTECTIVE SPECIALIST,1820.0,65605.06
3,BAKER,DENISE,MANHATTAN,ADM MANAGER-NON-MGRL,1820.0,105447.68
4,HARVIN,SHANGO,BRONX,YOUTH DEVELOPMENT SPECIALIST,1820.0,56771.73
5,SIDDIQ,JANNA,BRONX,CHILD PROTECTIVE SPECIALIST,1820.0,65567.23


## Part 1: mean, median, mode

In [79]:
mean = df['Regular Gross Paid'].mean()
median = df['Regular Gross Paid'].median()
mode = df['Regular Gross Paid'].mode()[0]

print("The mean payroll salary is", mean, "\nThe median is", median, "\nThe mode is", mode)

The mean payroll salary is 81982.62348630908 
The median is 74132.3 
The mode is 153923.33


Repeat the previous step using only the Python standard library (“the hard way” - not using pandas, a spreadsheet program, etc).

Hint: Use a dictionary to keep track of value counts.

In [80]:
# mean

mean = sum(df['Regular Gross Paid'])/len(df['Regular Gross Paid'])

print('Mean is: ', mean)

Mean is:  81982.62348630906


In [81]:
# median

def calculate_median(data):
    sorted_data = sorted(data)
    n = len(sorted_data)
    mid = n // 2

    if n % 2 == 1: 
        return sorted_data[mid]
    else:          
        return (sorted_data[mid - 1] + sorted_data[mid]) / 2

median = calculate_median(df['Regular Gross Paid'])

print('median is: ', median)

median is:  74132.3


In [82]:
# mode

def calculate_mode(data):
    counts = {}
    for value in data:
        counts[value] = counts.get(value, 0) + 1
    max_count = max(counts.values())
    modes = [key for key, count in counts.items() if count == max_count]
    return modes if len(modes) > 1 else modes[0]

mode = calculate_mode(df['Regular Gross Paid'])
print('mode is: ', mode)

mode is:  153923.33


## Visualization

In [83]:
borough = df[["Work Location Borough", "Regular Gross Paid"]].groupby("Work Location Borough").mean().reset_index()
borough

,Work Location Borough,Regular Gross Paid
0,ALBANY,84692.175455
1,BRONX,76769.134478
2,BROOKLYN,80018.269064
3,DELAWARE,83719.193788
4,DUTCHESS,106396.550435
5,GREENE,67499.010000
6,MANHATTAN,84091.468176
7,NASSAU,60088.478500
8,ORANGE,65311.630000
9,OTHER,96840.211613


In [84]:
def sparkline(data):
    # Define Unicode block characters for a sparkline
    spark_chars = '▁▂▃▄▅▆▇█'
    min_val = min(data)
    max_val = max(data)
    
    # Normalize data to fit into the sparkline range
    def scale(value):
        if max_val - min_val == 0:
            return len(spark_chars) - 1  # All values are the same
        return int((value - min_val) / (max_val - min_val) * (len(spark_chars) - 1))
    
    # Generate sparkline characters
    return ''.join(spark_chars[scale(value)] for value in data)


In [85]:
# Extract the 'Regular Gross Paid' values from each borough
values = borough["Regular Gross Paid"]

# Generate the sparkline for the values
sparkline_str = sparkline(values)

# Print each borough's mean with the sparkline
for borough, value, spark in zip(
    borough["Work Location Borough"], values, sparkline_str
):
    print(f"{borough}: {value} {spark}")


ALBANY: 84692.17545454545 ▄
BRONX: 76769.13447761194 ▃
BROOKLYN: 80018.26906421331 ▄
DELAWARE: 83719.19378787879 ▄
DUTCHESS: 106396.55043478262 █
GREENE: 67499.01 ▂
MANHATTAN: 84091.46817562221 ▄
NASSAU: 60088.478500000005 ▁
ORANGE: 65311.63 ▁
OTHER: 96840.21161290322 ▆
PUTNAM: 73738.70424242425 ▃
QUEENS: 81325.47684847495 ▄
RICHMOND: 88597.66521972317 ▅
SCHOHARIE: 98667.8772 ▆
SULLIVAN: 87573.29174311928 ▅
ULSTER: 88554.10097222222 ▅
WESTCHESTER: 85703.21344467641 ▄
